This material should help you get the ideas clearer from the first meeting:

In [ ]:
names=["Tomás", "Pauline", "Pablo", "Bjork","Alan","Juana"]
woman=[False,True,False,False,False,True]
ages=[32,33,28,30,32,27]
country=["Chile", "Senegal", "Spain", "Norway","Peru","Peru"]
education=["Bach", "Bach", "Master", "PhD","Bach","Master"]

# now in a dict:
data={'name':names, 'age':ages, 'girl':woman,'born In':country, 'degree':education}

#now into a DF
import pandas as pd

friends=pd.DataFrame.from_dict(data)
# seeing it:
friends

The result is what you expected, but you need to be sure of what data structure you have:

In [ ]:
#what is it?
type(friends)

In [ ]:
#this is good
friends.age

In [ ]:
#what is it?
type(friends.age)

In [ ]:
#this is good
friends['age'] #this is the same as friends.age

In [ ]:
#what is it?
type(friends['age'])

In [ ]:
#this is bad
friends.iloc[['age']]

In [ ]:
#this is bad
friends.loc[['age']]

In [ ]:
#this is bad
friends['age','born In']

In [ ]:
#this is good
friends[['age','born In']]

In [ ]:
# what is it?
type(friends[['age','born In']])

In [ ]:
#this is bad
friends.'born In'

In [ ]:
#this is good
friends.loc[:,['age','born In']]

In [ ]:
type(friends.loc[:,['age','born In']])

In [ ]:
#this is bad
friends.loc[:,['age':'born In']] #loc uses names, iloc uses positions

In [ ]:
#this is bad
friends.iloc[:,['age','born In']]

In [ ]:
# this is good (but different)
friends.iloc[:,1:4]

In [ ]:
# what is it?
type(friends.iloc[:,1:4])

In [ ]:
# this is good
friends.iloc[:,[1,3]]

In [ ]:
#what is it?
type(friends.iloc[:,[1,3]])

In [ ]:
friends[friends.age>30]

Some people like coding with the filter language:

In [ ]:
# 
filter1=friends.age>30
friends[filter1]

In [ ]:
friends.where(filter1)

In [ ]:
filter1a='age>30'
friends.query(filter1a) #query works on subset of data frame, uses most recent data frame in memory

In [ ]:
isinstance(friends[filter1], pd.DataFrame), \ #backslash lets you break up one giant line to make it more readable
isinstance(friends.where(filter1), pd.DataFrame), \
isinstance(friends.query(filter1a), pd.DataFrame)

When you have Boolean values (True/False) you can simplify:

In [ ]:
#from:
friends[friends.girl==False]

In [ ]:
# to...
friends[~friends.girl]

You can have two filters:

In [ ]:
# this will not work
friends[~friends.girl & friends.degree=='Bach']

In [ ]:
# this will (with parentheses)
friends[(~friends.girl) & (friends.degree=='Bach')]

Other times you want a values once a filter was applied:

In [ ]:
# youngest male:
friends[(~friends.girl) & (friends.age.min())] # this is wrong!

In [ ]:
friends[(~friends.girl) & (friends.age==friends.age.min())] # this is wrong too!

In [ ]:
friends.age.min()

You got empty answer because there is no man aged 27.

In [ ]:
# this is correct
friends[~friends.girl].age.min()

Once you know the right age, you have to put it in the right place:

In [ ]:
friends[friends.age==friends[~friends.girl].age.min()] #not great because if there were a girl with age of 28 she would show up here

In [ ]:
# or
friends.where(friends.age==friends[~friends.girl].age.min())

In [ ]:
# or
friends.where(friends.age==friends[~friends.girl].age.min()).dropna() #dropna gets rid of rows with missing values

The problem is that 'friends' are not subset and the age keeps being that of the youngest woman:

In [ ]:
# bad:
friends.where(~friends.girl).where(friends.age==friends.age.min())

That's the advantage of **query**:

In [ ]:
friends.query('~girl').query('age==age.min()') #note the quotes around the query function

In [ ]:
#but

students=friends.copy()

students.where(~students.girl,inplace=True) #real subset...inplace immediately changes the original dataframe (students in this case)
students.where(students.age==students.age.min())

Let's vary the data a little:

In [ ]:
names=["Tomás", "Pauline", "Pablo", "Bjork","Alan","Juana"]
woman=[False,True,False,False,False,True]
ages=[32,28,28,30,32,27]
country=["Chile", "Senegal", "Spain", "Norway","Peru","Peru"]
education=["Bach", "Bach", "Master", "PhD","Bach","Master"]

# now in a dict:
data={'name':names, 'age':ages, 'girl':woman,'born In':country, 'degree':education}

#now into a DF
import pandas as pd

friends2=pd.DataFrame.from_dict(data)
# seeing it:
friends2

There is a girl with the same age as the youngest boy, then:

In [48]:
friends2.where(friends2.age==friends2[~friends2.girl].age.min()).dropna()

,name,age,girl,born In,degree
1,Pauline,28.0,1.0,Senegal,Bach
2,Pablo,28.0,0.0,Spain,Master


We need a previous strategy:

In [ ]:
# bad implementation:
friends2.where(friends2.age==friends2[~friends2.girl].age.min() & friends2.girl==False).dropna()

In [ ]:
# bad implementation:
friends2.where(friends2.age==friends2[~friends2.girl].age.min() & ~friends2.girl).dropna()

In [49]:
# just parentheses to make it work!
friends2.where((friends2.age==friends2[~friends2.girl].age.min()) & (~friends2.girl)).dropna()

,name,age,girl,born In,degree
2,Pablo,28.0,0.0,Spain,Master


This one still works!

In [50]:
friends2.query('~girl').query('age==age.min()')

,name,age,girl,born In,degree
2,Pablo,28,False,Spain,Master


In [51]:
students2=friends2.copy()

students2.where(~students2.girl,inplace=True) #real subset
students2.where(students2.age==students2.age.min()).dropna()

,name,age,girl,born In,degree
2,Pablo,28.0,0.0,Spain,Master
